# Parallelize a flat histogram simulation

A grand canonical ensemble transition-matrix Monte Carlo simulation is parallelized by dividing the macrostate ranges of each processor into overlapping windows.

As an exercise, compare the macrostate probability distribution output for all windows with the individual windows in the files (crit#.txt).
Attempt to unify these individual macrostate probability distributions.

In [1]:
import feasst as fst
import pyfeasst

def mc(thread, mn, mx):
    steps_per=int(1e5)
    mc = fst.MakeMonteCarlo()
    mc.add(fst.MakeConfiguration(fst.args({"cubic_box_length": "8",
                                           "particle_type0": fst.install_dir() + "/forcefield/lj.fstprt"})))
    mc.add(fst.MakePotential(fst.MakeLennardJones()))
    mc.add(fst.MakePotential(fst.MakeLongRangeCorrections()))
    mc.set(fst.MakeThermoParams(fst.args({"beta": str(1./1.5),
                                          "chemical_potential": "-2.352321"})))
    mc.set(fst.MakeFlatHistogram(
        fst.MakeMacrostateNumParticles(
              fst.Histogram(fst.args({"width": "1", "max": str(mx), "min": str(mn)}))),
        fst.MakeTransitionMatrix(fst.args({"min_sweeps": "10"}))))
    mc.add(fst.MakeTrialTranslate(fst.args({"weight": "1.", "tunable_param": "1."})))
    mc.add(fst.MakeTrialTransfer(fst.args({"particle_type": "0", "weight": "4"})))
    mc.add(fst.MakeCheckEnergyAndTune(fst.args({"steps_per": str(steps_per), "tolerance": "0.0001"})))
    mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": str(steps_per), "file_name": "clones" + str(thread)})))
    mc.add(fst.MakeCriteriaUpdater(fst.args({"steps_per": str(steps_per)})))
    mc.add(fst.MakeCriteriaWriter(fst.args({
        "steps_per": str(steps_per),
        "file_name": "clones" + str(thread) + "_crit.txt"})))
    return mc

windows=fst.WindowExponential(fst.args({
  "alpha": "2",
  "num": "2",
  "maximum": "50",
  "extra_overlap": "2"})).boundaries()
print(windows)

clones = fst.Clones()
for proc, win in enumerate(windows):
    clones.add(mc(proc, win[0], win[1]))
clones.initialize_and_run_until_complete()
#print(clones.ln_prob().values())
open('clones.fst', 'w').write(clones.serialize())

((0, 35), (33, 50))


175281

You may post-process the simulations from the rst file.

In [2]:
clones2 = fst.Clones().deserialize(pyfeasst.read_checkpoint('clones.fst'))
# import matplotlib.pyplot as plt
# plt.plot(clones2.ln_prob().values())

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!